In [1]:
!pip install requests

In [2]:
import requests
import datetime
from urllib.parse import urlencode

In [3]:
import base64

In [4]:
client_id = '9ec14c18b8114a569770be752b524b7e'
client_secret = '6d13d61e65a2418697e225936274c7bf'

In [40]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    auth_token= "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args,**kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_cred = f"{client_id}:{client_secret}"
        client_cred_b64 = base64.b64encode(client_cred.encode())
        return client_cred_b64.decode()
        
    def get_token_header(self):
        client_cred_b64 = self.get_client_credentials()
        return{
             "Authorization" : f"Basic {client_cred_b64}"
        }
    def get_token_data(self):
        return{
            "grant_type": "client_credentials"
        }
    
    def Authorization(self):
        auth_token = self.auth_token
        token_data = self.get_token_data()
        token_header = self.get_token_header()
        r = requests.post(auth_token,data=token_data, headers=token_header)
        if r.status_code not in range(200,299):
            raise Exception("could not authenticate client")
            #return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.Authorization()
            return self.get_access_token()
        elif token == None:
            return self.get_access_token()
        return token
    
    def get_resources_header(self):
        access_token = self.get_access_token()
        header = {
            "Authorization": f"Bearer {access_token}"
        }
        return header
    
    def get_resources(self, _id, resource_type="albums", version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{_id}"
        header = self.get_resources_header()
        r = requests.get(endpoint, headers=header)
        if r.status_code not in range(200,299):
            return{}
        return r.json()
    
    def get_album(self,_id):
        return self.get_resources(_id,resource_type='albums')
    
    def get_artist(self,_id):
        return self.get_resources(_id,resource_type='artists')
    
    
    def search(self, query, search_type='artists'):
        access_token = self.get_access_token()
        header = {
            "Authorization": f"Bearer {access_token}"
        }
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": "Time", "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=header)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
        
        

In [41]:
spotify = SpotifyAPI(client_id, client_secret)

In [42]:
spotify.search("shake it off", search_type="track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Time&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3tJoFztHeIJkJWMrx0td2f'},
       'href': 'https://api.spotify.com/v1/artists/3tJoFztHeIJkJWMrx0td2f',
       'id': '3tJoFztHeIJkJWMrx0td2f',
       'name': 'Moneybagg Yo',
       'type': 'artist',
       'uri': 'spotify:artist:3tJoFztHeIJkJWMrx0td2f'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',
      'DM'

In [45]:
spotify.get_artist("3TVXtAsR1Inumwj472S9r4")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
 'followers': {'href': None, 'total': 57434946},
 'genres': ['canadian hip hop',
  'canadian pop',
  'hip hop',
  'pop rap',
  'rap',
  'toronto rap'],
 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
 'id': '3TVXtAsR1Inumwj472S9r4',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb9e46a78c5cd2f7a8e7669980',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051749e46a78c5cd2f7a8e7669980',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1789e46a78c5cd2f7a8e7669980',
   'width': 160}],
 'name': 'Drake',
 'popularity': 100,
 'type': 'artist',
 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}